## A simple swarm

In [1]:
%load_ext autoreload 
%autoreload 2
import sys
sys.tracebacklimit = 0

In [2]:
from core.agent import *
from core.world import * 
from core.map import *
import numpy as np
from core.render import * 

pygame-ce 2.4.1 (SDL 2.28.5, Python 3.11.9)


In [3]:
def initialize_swarm(world : World):
    swarm = [Agent() for i in range(1000)]
    for agent in swarm:
        world.add_agent(agent)
    swarm = initialize_positions_randomly(world, swarm)
    

In [4]:
world = World(dims = (100, 100),
              swarm_initialzier= initialize_swarm,
              resource_generator= RandomMapGenerator(),
              energy_model=EnergyModel()
              )
world.reset()

In [5]:
# Test movements by doing random actions
def update():
    world.update()
    # Comms protocol (receiving)
    for agent in world.agents: 
        received_messages = agent.get_messages()
        for msg in received_messages: 
            agent.add_relation(msg.sender, msg.message)
        agent.clear_messages()

    for agent in world.agents:
        action_choice = np.random.randint(0, 4)
        choice = np.random.randint(1, 5)

        match (action_choice): 
            case 1: agent.move(choice)
            case 2: agent.pick_up(choice)
            case 3: agent.put_down(choice)
    
    # Comms protocol (sending )
    for agent in world.agents:
        neighbors = agent.agents_in_range 

        # Send agents
        if len (neighbors) > 0:
            choice = np.random.choice(neighbors)
            tgt_agent = world.get_agent(choice)
            agent.talk_to(tgt_agent)

update()

In [6]:
render_world(world, (800, 800), update_fn=update, delay_s=0)

## Verify certain things about the simulation

In [7]:
swarm = world.agents

In [8]:
# Validate the observation space provided is sensible
obs = swarm[0].local_observation

print(obs.nearby_agents, obs.nearby_agents.shape)
print(obs.resource_types, obs.resource_types.shape)

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]] (7, 7)
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]] (7, 7)


In [9]:
# Validate that all swarm agents landed in different positions
positions = set()
for agent in swarm:
    pos = agent.current_position
    positions.add((pos[0], pos[1]))

print(len(positions), len(world.agents))

1000 1000


In [10]:
# Check for resources
rsrc_qty = world._resource_map._resource_quantity_map
qtys = []

for i in range(rsrc_qty.shape[0]):
    for j in range(rsrc_qty.shape[1]):
        if rsrc_qty[i, j] > 0:
            qtys.append(rsrc_qty[i, j])

print(qtys)

[0.9234200185884873, 2.804297891491038, 5.664825941976331, 6.4623227898368665, 4.7602893544172105, 0.050618414429145275, 4.360162760547318, 4.644106302743007, 0.05081133046581687, 4.364345606566936, 3.6928552066058242, 5.192448995786768, 5.30787511884904, 0.2135510399081264, 0.05726438217401064, 5.500935519365128, 4.9429268270559135, 2.7263944510558726, 0.694064832507582, 1.0166291749758227, 3.0449396529139756, 5.571646323381626, 1.2536622250494358, 6.213101013080491, 3.3343763380567566, 3.812897013697475, 4.507424536848931, 5.297475795169361, 2.1757637044125286, 10.56236439421641, 4.471870487359046, 3.447227155259112, 11.06695453360859, 6.733442337009051, 5.120681586385389, 8.430046543507014, 5.037094829803043, 5.220442379179751, 8.04059599534255, 2.9200485219490186, 1.4022090738568025, 8.331606638152447, 4.175883574664579, 4.17063331570482, 3.0688591372506355, 4.773591016530415, 5.7130469896385705, 5.941077396780166, 4.018731742323553, 4.48038398490335, 4.762761013422499, 2.397427311

In [11]:
# Check for agent state
state = swarm[0]._current_state.inventory
print(state)


[Resource(type=5, quantity=1.0), Resource(type=5, quantity=1.0), Resource(type=5, quantity=1.0), Resource(type=5, quantity=1.0), Resource(type=5, quantity=1.0), Resource(type=5, quantity=1.0), Resource(type=5, quantity=1.0), Resource(type=5, quantity=0.5752760537344495), Resource(type=5, quantity=1.0)]


# Perftest

In [12]:
import cProfile
def stress_test():
    for _ in range(0, 1000):
        update()

In [16]:
world.reset()
cProfile.run('stress_test()', sort = 'time')

         71402639 function calls in 31.714 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  2000000    2.709    0.000    2.709    0.000 {method 'randint' of 'numpy.random.mtrand.RandomState' objects}
  1000000    2.372    0.000    2.716    0.000 observation.py:20(neighbors)
     1000    1.787    0.002   31.902    0.032 254406432.py:2(update)
     1000    1.754    0.002    3.975    0.004 world.py:88(_update_movement)
   615655    1.644    0.000    3.948    0.000 {method 'choice' of 'numpy.random.mtrand.RandomState' objects}
  1000000    1.604    0.000    2.796    0.000 world.py:128(_get_nearby_agents)
  1000000    1.590    0.000    2.756    0.000 world.py:148(_get_nearby_resources)
  6536052    1.420    0.000    1.420    0.000 {method 'copy' of 'numpy.ndarray' objects}
     1000    1.266    0.001   17.656    0.018 world.py:51(update)
   493671    1.075    0.000    1.212    0.000 map.py:43(subtract_resource)
  6536052    1.0

In [ ]:
cProfile.run('world.update()', sort = 'time')